In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import re
import math
from tqdm import tqdm

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('indonesian'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
import gdown

file_id = "1ysv88LcYDyjYrSrC_Fke4JThiv2BuP9y"
url = f"https://drive.google.com/uc?id={file_id}"

gdown.download(url, "data.xml", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1ysv88LcYDyjYrSrC_Fke4JThiv2BuP9y
From (redirected): https://drive.google.com/uc?id=1ysv88LcYDyjYrSrC_Fke4JThiv2BuP9y&confirm=t&uuid=5d8faae1-ca30-4e72-8832-3d66f430de6c
To: d:\IRD\praktikum\pertemuan9\data.xml
100%|██████████| 2.56M/2.56M [00:45<00:00, 55.9kB/s]


'data.xml'

In [3]:
df = pd.read_xml('data.xml')
df.head()

,id,sumber,tanggal,kategori,judul,isi,link,jumlahkata
0,1,kompas.com,2015/07/01,Teknologi,"Ponsel Huawei Honor 4C Dibanderol Rp 2,2 Juta","JAKARTA, KOMPAS.com Ponsel Android Huawei Hon...",http://tekno.kompas.com/read/xml/2015/07/01/18...,315
1,2,kompas.com,2015/07/01,Teknologi,Asosiasi: RPP E-commerce Tidak Sesuai Hasil Di...,"JAKARTA, KOMPAS.com - Sejak 2013, wacana tenta...",http://tekno.kompas.com/read/xml/2015/07/01/16...,419
2,3,kompas.com,2015/07/01,Teknologi,"Pemesan ""iPhone Jadi Sabun"" Karyawan Pesaing L...","JAKARTA, KOMPAS.com Danis Darusman, pelanggan ...",http://tekno.kompas.com/read/xml/2015/07/01/15...,265
3,4,kompas.com,2015/07/01,Teknologi,"""Autofeather Failure"", Momok bagi Pesawat Bali...",KOMPAS.com Salah satu momok yang dihadapi dala...,http://tekno.kompas.com/read/xml/2015/07/01/14...,481
4,5,kompas.com,2015/07/01,Teknologi,Laptop Bezel Tertipis di Dunia Masuk Indonesia,"JAKARTA, KOMPAS.com - Resmi diperkenalkan pada...",http://tekno.kompas.com/read/xml/2015/07/01/14...,227


In [4]:
df.drop(columns=['sumber', 'tanggal', 'link'], inplace=True)
df.head()

,id,kategori,judul,isi,jumlahkata
0,1,Teknologi,"Ponsel Huawei Honor 4C Dibanderol Rp 2,2 Juta","JAKARTA, KOMPAS.com Ponsel Android Huawei Hon...",315
1,2,Teknologi,Asosiasi: RPP E-commerce Tidak Sesuai Hasil Di...,"JAKARTA, KOMPAS.com - Sejak 2013, wacana tenta...",419
2,3,Teknologi,"Pemesan ""iPhone Jadi Sabun"" Karyawan Pesaing L...","JAKARTA, KOMPAS.com Danis Darusman, pelanggan ...",265
3,4,Teknologi,"""Autofeather Failure"", Momok bagi Pesawat Bali...",KOMPAS.com Salah satu momok yang dihadapi dala...,481
4,5,Teknologi,Laptop Bezel Tertipis di Dunia Masuk Indonesia,"JAKARTA, KOMPAS.com - Resmi diperkenalkan pada...",227


In [5]:
df['kategori'].unique()

array(['Teknologi', 'Bisnis Ekonomi', 'Nasional', 'Olahraga', 'Travel',
       'Bola', 'Otomotif', 'Lifestyle'], dtype=object)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502 entries, 0 to 1501
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1502 non-null   int64 
 1   kategori    1502 non-null   object
 2   judul       1502 non-null   object
 3   isi         1502 non-null   object
 4   jumlahkata  1502 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 58.8+ KB


In [7]:
df.isnull().sum()

id            0
kategori      0
judul         0
isi           0
jumlahkata    0
dtype: int64

In [8]:
df.duplicated().sum()

np.int64(0)

### Tahap Preprocessing Data

In [9]:
def caseFolding(text):
    text = text.lower()
    return text

contoh = "Terima kasih, Kak! Kamu sangat baik sekali kepadaku hari ini."
print(f'original: {contoh}')
print(f'case folding: {caseFolding(contoh)}')

original: Terima kasih, Kak! Kamu sangat baik sekali kepadaku hari ini.
case folding: terima kasih, kak! kamu sangat baik sekali kepadaku hari ini.


In [10]:
def punctuationRemoval(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

contoh = "Terima kasih, kak! Kamu sangat baik sekali kepadaku hari ini."
print(f'original: {contoh}')
print(f'Punctuation Removal Text: {punctuationRemoval(contoh)}')

original: Terima kasih, kak! Kamu sangat baik sekali kepadaku hari ini.
Punctuation Removal Text: Terima kasih kak Kamu sangat baik sekali kepadaku hari ini


In [11]:
stop_words

{'ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [12]:
def remove_stopwords(text):
    tokens = text.split()
    filtered = []

    # Loop setiap kata
    for word in tokens:
        if word not in stop_words:
            filtered.append(word)

    # Menggabungkan kembali kata-kata menjadi kalimat
    cleaned_text = " ".join(filtered)
    return cleaned_text

contoh_teks = "Saya suka sama kamu, tapi kamu suka dia"
print("Teks Asli: ", contoh_teks)
print("Setelah Proses:", remove_stopwords(contoh_teks))

Teks Asli:  Saya suka sama kamu, tapi kamu suka dia
Setelah Proses: Saya suka kamu, suka


In [13]:
df['clean'] = df['isi'].apply(caseFolding)
df['clean'] = df['clean'].apply(punctuationRemoval)
df['clean'] = df['clean'].apply(remove_stopwords)
df.head()

,id,kategori,judul,isi,jumlahkata,clean
0,1,Teknologi,"Ponsel Huawei Honor 4C Dibanderol Rp 2,2 Juta","JAKARTA, KOMPAS.com Ponsel Android Huawei Hon...",315,jakarta kompascom ponsel android huawei honor ...
1,2,Teknologi,Asosiasi: RPP E-commerce Tidak Sesuai Hasil Di...,"JAKARTA, KOMPAS.com - Sejak 2013, wacana tenta...",419,jakarta kompascom 2013 wacana rancangan peratu...
2,3,Teknologi,"Pemesan ""iPhone Jadi Sabun"" Karyawan Pesaing L...","JAKARTA, KOMPAS.com Danis Darusman, pelanggan ...",265,jakarta kompascom danis darusman pelanggan laz...
3,4,Teknologi,"""Autofeather Failure"", Momok bagi Pesawat Bali...",KOMPAS.com Salah satu momok yang dihadapi dala...,481,kompascom salah momok dihadapi pesawat turbopr...
4,5,Teknologi,Laptop Bezel Tertipis di Dunia Masuk Indonesia,"JAKARTA, KOMPAS.com - Resmi diperkenalkan pada...",227,jakarta kompascom resmi diperkenalkan ajang co...


### Tahap TF

In [14]:
def compute_word_frequency(text):
    words = word_tokenize(text)
    frequency_table = {}

    for word in words:
        if word in frequency_table:
            frequency_table[word] += 1
        else:
            frequency_table[word] = 1

    return frequency_table

compute_word_frequency(df['clean'][0])

{'jakarta': 1,
 'kompascom': 1,
 'ponsel': 5,
 'android': 2,
 'huawei': 3,
 'honor': 4,
 '4c': 5,
 'resmi': 1,
 'meluncur': 1,
 'pasar': 1,
 'indonesia': 2,
 'rabu': 1,
 '172015': 1,
 'perusahaan': 2,
 'tiongkok': 1,
 'merancangnya': 2,
 'kelas': 1,
 'menengah': 1,
 'banderol': 1,
 'harga': 1,
 'rp': 1,
 '23': 1,
 'jutakami': 1,
 'pengguna': 3,
 'usia': 1,
 '1835': 1,
 'prosesor': 3,
 'octa': 1,
 'core': 1,
 'kinerja': 1,
 'mulus': 1,
 'bermain': 1,
 'game': 1,
 'lag': 1,
 'klaim': 1,
 'sales': 1,
 'manager': 1,
 'device': 1,
 'department': 1,
 'darren': 3,
 'peluncuran': 1,
 'jakartaponsel': 1,
 'berukuran': 1,
 '5': 2,
 'inci': 1,
 'kirin': 1,
 '620': 1,
 'berkecepatan': 1,
 '12': 1,
 'ghz': 1,
 'pembuatanya': 1,
 'hisilicon': 1,
 'semi': 1,
 'konduktor': 1,
 'milik': 1,
 'huaweihonor': 1,
 'dibekali': 1,
 'memori': 1,
 'ram': 1,
 '2': 1,
 'gb': 2,
 'rom': 1,
 '8': 1,
 'media': 1,
 'penyimpanan': 1,
 'aplikasi': 1,
 'sistem': 2,
 'operasi': 1,
 '44': 1,
 'kitkat': 1,
 'lollipop': 1,


In [15]:
df['word_frequency'] = df['clean'].apply(compute_word_frequency)
df.head()

,id,kategori,judul,isi,jumlahkata,clean,word_frequency
0,1,Teknologi,"Ponsel Huawei Honor 4C Dibanderol Rp 2,2 Juta","JAKARTA, KOMPAS.com Ponsel Android Huawei Hon...",315,jakarta kompascom ponsel android huawei honor ...,"{'jakarta': 1, 'kompascom': 1, 'ponsel': 5, 'a..."
1,2,Teknologi,Asosiasi: RPP E-commerce Tidak Sesuai Hasil Di...,"JAKARTA, KOMPAS.com - Sejak 2013, wacana tenta...",419,jakarta kompascom 2013 wacana rancangan peratu...,"{'jakarta': 2, 'kompascom': 1, '2013': 1, 'wac..."
2,3,Teknologi,"Pemesan ""iPhone Jadi Sabun"" Karyawan Pesaing L...","JAKARTA, KOMPAS.com Danis Darusman, pelanggan ...",265,jakarta kompascom danis darusman pelanggan laz...,"{'jakarta': 1, 'kompascom': 1, 'danis': 12, 'd..."
3,4,Teknologi,"""Autofeather Failure"", Momok bagi Pesawat Bali...",KOMPAS.com Salah satu momok yang dihadapi dala...,481,kompascom salah momok dihadapi pesawat turbopr...,"{'kompascom': 1, 'salah': 2, 'momok': 1, 'diha..."
4,5,Teknologi,Laptop Bezel Tertipis di Dunia Masuk Indonesia,"JAKARTA, KOMPAS.com - Resmi diperkenalkan pada...",227,jakarta kompascom resmi diperkenalkan ajang co...,"{'jakarta': 2, 'kompascom': 1, 'resmi': 1, 'di..."


In [16]:
# jumlah total dokumen
N = len(df)

# menghitung document frequency untuk setiap term
document_frequencies = {}

for word_freq in df['word_frequency']:
    for term in word_freq.keys():
        if term in document_frequencies:
            document_frequencies[term] += 1
        else:
            document_frequencies[term] = 1

# menyimpan kedalam dictionary
idf = {}

# menghitung IDF untuk setiap term
for word, df_t in document_frequencies.items():
    idf[word] = math.log(N / (1 + df_t))

# cek bebeerapa contoh nilai IDF
list(idf.items())[:10]

[('jakarta', 0.8656634381772226),
 ('kompascom', 2.317340558559965),
 ('ponsel', 4.17905861639493),
 ('android', 5.117328254987861),
 ('huawei', 5.522793363096025),
 ('honor', 5.705114919889979),
 ('4c', 5.928258471204189),
 ('resmi', 2.518762286727339),
 ('meluncur', 4.82964618253608),
 ('pasar', 2.957844005634488)]

In [17]:
# Menghitung TF-IDF untuk setiap dokumen
def compute_tf_idf(word_freq, idf_dict):
    tf_idf = {}
    for word, tf in word_freq.items():
        if word in idf_dict:
            # TF-IDF - TF * IDF
            tf_idf[word] = tf * idf_dict[word]
    return tf_idf

tf_idf_list = []

for freq in df['word_frequency']:
    tf_idf_value = compute_tf_idf(freq, idf)
    tf_idf_list.append(tf_idf_value)

df['tf_idf'] = tf_idf_list
df.head()

,id,kategori,judul,isi,jumlahkata,clean,word_frequency,tf_idf
0,1,Teknologi,"Ponsel Huawei Honor 4C Dibanderol Rp 2,2 Juta","JAKARTA, KOMPAS.com Ponsel Android Huawei Hon...",315,jakarta kompascom ponsel android huawei honor ...,"{'jakarta': 1, 'kompascom': 1, 'ponsel': 5, 'a...","{'jakarta': 0.8656634381772226, 'kompascom': 2..."
1,2,Teknologi,Asosiasi: RPP E-commerce Tidak Sesuai Hasil Di...,"JAKARTA, KOMPAS.com - Sejak 2013, wacana tenta...",419,jakarta kompascom 2013 wacana rancangan peratu...,"{'jakarta': 2, 'kompascom': 1, '2013': 1, 'wac...","{'jakarta': 1.7313268763544452, 'kompascom': 2..."
2,3,Teknologi,"Pemesan ""iPhone Jadi Sabun"" Karyawan Pesaing L...","JAKARTA, KOMPAS.com Danis Darusman, pelanggan ...",265,jakarta kompascom danis darusman pelanggan laz...,"{'jakarta': 1, 'kompascom': 1, 'danis': 12, 'd...","{'jakarta': 0.8656634381772226, 'kompascom': 2..."
3,4,Teknologi,"""Autofeather Failure"", Momok bagi Pesawat Bali...",KOMPAS.com Salah satu momok yang dihadapi dala...,481,kompascom salah momok dihadapi pesawat turbopr...,"{'kompascom': 1, 'salah': 2, 'momok': 1, 'diha...","{'kompascom': 2.317340558559965, 'salah': 3.19..."
4,5,Teknologi,Laptop Bezel Tertipis di Dunia Masuk Indonesia,"JAKARTA, KOMPAS.com - Resmi diperkenalkan pada...",227,jakarta kompascom resmi diperkenalkan ajang co...,"{'jakarta': 2, 'kompascom': 1, 'resmi': 1, 'di...","{'jakarta': 1.7313268763544452, 'kompascom': 2..."


In [18]:
# pilih dokumen berdasarkan id berita
id_berita = 1
row = df[df['id'] == id_berita].iloc[0]

# Ambil data TF dan IDF dari dokumen tersebut
tf_data = row['word_frequency']
tfidf_data = row['tf_idf']

rows = []
for term, tf in tf_data.items():
    idf_value = idf.get(term, 0)
    tfidf_value = tfidf_data.get(term, 0)
    rows.append({
        'term': term,
        'tf':tf,
        'idf': round(idf_value, 4),
        'tfidf': round(tfidf_value, 4)
    })

df_tfidf_detail = pd.DataFrame(rows)

# Mengurutkan dari nilai tfidf terbesar
df_tfidf_detail = df_tfidf_detail.sort_values(by='tfidf', ascending=False)

print(f"Hasil TF-IDF untuk id berita = {id_berita}\n")
print(df_tfidf_detail.to_string(index=False))

Hasil TF-IDF untuk id berita = 1

             term  tf    idf   tfidf
               4c   5 5.9283 29.6413
            honor   4 5.7051 22.8205
           ponsel   5 4.1791 20.8953
           darren   3 6.6214 19.8642
           huawei   3 5.5228 16.5684
         prosesor   3 4.7496 14.2488
     merancangnya   2 6.6214 13.2428
            photo   2 6.2159 12.4319
          refocus   2 5.9283 11.8565
         snapshot   2 5.9283 11.8565
       megapiksel   2 5.9283 11.8565
     penjualannya   2 5.9283 11.8565
            ultra   2 5.7051 11.4102
             fast   2 5.7051 11.4102
           selfie   2 5.7051 11.4102
         pengguna   3 3.7882 11.3646
               gb   2 5.5228 11.0456
             mode   2 5.3686 10.7373
         memotret   2 5.2351 10.4702
          android   2 5.1173 10.2347
           kamera   2 4.3701  8.7402
            fitur   2 4.0957  8.1914
             foto   2 4.0565  8.1129
      pembuatanya   1 6.6214  6.6214
        hisilicon   1 6.6214  6.6214
    

### Ranked Retrival

In [19]:
# Mengumpulkan kata dalam setiap dokumen
all_terms_set = set() # Menyiapkan kedalam bentuk set

for _, row in df.iterrows():
    # Mengambil kata TF-IDF dokumen
    terms_in_doc = row['tf_idf'].keys()
    # Menambahkan ke kumpulan semua kata
    all_terms_set.update(terms_in_doc)

all_terms = sorted(all_terms_set) # Mengurutkan

In [20]:
all_terms[:10]

['0', '00', '00004', '001', '002', '003', '00304', '004', '005', '006']

In [21]:
# Membuat matriks TF-IDF menggunakan Numpy
print("\nMembuat matriks TF-IDF untuk seluruh dokumen dokumen")
doc_ids = df['id'].astype(int).tolist()

doc_vectors = np.array([
    [row['tf_idf'].get(term, 0) for term in all_terms]
    for _, row in tqdm(df.iterrows(), total=len(df))
])


Membuat matriks TF-IDF untuk seluruh dokumen dokumen


100%|██████████| 1502/1502 [01:46<00:00, 14.04it/s]


In [22]:
# Membuat tabel tf-idf
print("\nMembuat tabel TF-IDF")
df_tfidf_table = pd.DataFrame(doc_vectors.T, index=all_terms, columns=[f"d{doc_id}" for doc_id in doc_ids])
df_tfidf_table.index.name = "Term"


Membuat tabel TF-IDF


In [23]:
from IPython.display import display

display(df_tfidf_table.iloc[10200:10300])

,d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,...,d1493,d1494,d1495,d1496,d1497,d1498,d1499,d1500,d1501,d1502
Term,,,,,,,,,,,,,,,,,,,,,
kandidat,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
kandung,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
kandungadik,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
kandungan,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
kandungnya,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
kartikawati,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
kartina,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
kartoharjo,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
# 1. pilih kolom 'd1' dari Dataframe
series_d1 = df_tfidf_table['d1']

# 2. Terapkan Boolean Indexing: Filter Series_d1 untuk nilai yang lebih besar dari 0.0
nilai_diatas_nol_d1 = series_d1[series_d1 > 0.0]

# 3. Tampilkan hasilnya
print("--- Bobot TF-IDF (d1) diatas 0.0 ---")
display(nilai_diatas_nol_d1)

--- Bobot TF-IDF (d1) diatas 0.0 ---


Term
10          2.651114
12          2.596054
13          3.731034
172015      2.062279
1835        6.215941
              ...   
usb         6.621406
usia        4.018716
utamanya    4.829646
wide        6.215941
wifi        5.522793
Name: d1, Length: 173, dtype: float64

### Persiapkan Vector dari contoh Query

In [25]:
# 1. Fungsi Preprocessing
def preprocess_query(query):
    # Case folding
    query = caseFolding(query)
    # Punctuation removal
    query = punctuationRemoval(query)
    # Remove stopwords
    query = remove_stopwords(query)
    return query

In [26]:
# 2. contoh query dan menerapkan fungsi preprocessing
query = "ekonomi indonesia pertumbuhan inflasi"
query = preprocess_query(query)

In [27]:
# 3. Kita jalankan fungsi tf kemudian disimpan dalam variabel q_tf
q_tf = compute_word_frequency(query)
print("TF Query:", q_tf)

TF Query: {'ekonomi': 1, 'indonesia': 1, 'pertumbuhan': 1, 'inflasi': 1}


In [28]:
# 4. gunakan variabel q_tf dan idf untuk menyimpan ke dalam q_tfidf
q_tfidf = compute_tf_idf(q_tf, idf)
print("TF-IDF Query:", q_tfidf)

TF-IDF Query: {'ekonomi': 3.0950451271479733, 'indonesia': 1.3983507697166448, 'pertumbuhan': 3.913355450661925, 'inflasi': 4.223510378965764}


In [29]:
# Membuat vektor query dengan panjang yang sama seperti vektor dokumen
query_vector = np.array([q_tfidf.get(term, 0) for term in all_terms])

print("Panjang vector query:", len(query_vector))

Panjang vector query: 24247


### Penerapan Coshine Similarity

In [ ]:
# 1. Membuat Fungsi untuk Cosine Similarity
def cosine_similarity_manual(vec_1, vec_2):
    dot_product = np.dot(vec_1, vec_2)                 #Perkalian titik
    norm_a = np.linalg.norm(vec_1)                     #Panjang vektor A
    norm_b = np.linalg.norm(vec_2)                     #Panjang vektor B
    
    if norm_a == 0 or norm_b == 0:                     #Untuk menghindari pembagian dengan nol
        return 0
    else:
        return dot_product / (norm_a * norm_b)

In [31]:
# 2. siapkan variable dictionary
similarities = []

for i, doc_vec in enumerate(doc_vectors):
    sim = cosine_similarity_manual(query_vector, doc_vec)
    similarities.append((df.iloc[i], sim))

# Urutkan dari similaritas tertinggi
ranked_results = sorted(similarities, key=lambda x: x[1], reverse=True)

# Tampilkan 10 Dokumen Teratas
print("\n=== Hasil Ranked Retrival (Top 10) ===")
for doc_id, sim in ranked_results[:10]:
    print(f"Doc {doc_id} -> similarity: {sim:.4f}")


=== Hasil Ranked Retrival (Top 10) ===
Doc id                                                              396
kategori                                             Bisnis Ekonomi
judul                                                       Ekonomi
isi               REPUBLIKA.CO.ID, JAKARTA -- Bank Indonesia mem...
jumlahkata                                                      227
clean             republikacoid jakarta bank indonesia mempredik...
word_frequency    {'republikacoid': 1, 'jakarta': 1, 'bank': 1, ...
tf_idf            {'republikacoid': 1.4480847753907833, 'jakarta...
Name: 395, dtype: object -> similarity: 0.4680
Doc id                                                              402
kategori                                             Bisnis Ekonomi
judul                                                       Ekonomi
isi               REPUBLIKA.CO.ID, JAKARTA -- Bank Indonesia men...
jumlahkata                                                      259
clean             rep

In [32]:
# Mengecek dokumen apakah relevant dengan query
doc_to_check = 396
row = df[df['id'] == doc_to_check]

if row.empty:
    print("ID Dokumen tidak ditemukan.")
else:
    print("\n=== Isi Berita (ID:", doc_to_check, ") ===")
    print(row['isi'].iloc[0])


=== Isi Berita (ID: 396 ) ===
REPUBLIKA.CO.ID, JAKARTA -- Bank Indonesia memprediksi laju inflasi pada akhir 2015 akan mencapai level 4,2 persen hingga 4,3 persen secara tahunan meskipun saat ini masih di atas tujuh persen."Kalau kita kihat akhir kuartal I dan II 2015 ini inflasinya di atas tujuh persen, namun akhir tahun akan ada di plus minus satu persen year on year (secara tahunan)," kata Gubernur BI Agus Martowardojo, Rabu (1/7). Inflasi Juni 2015, sebagaimana baru dirilis oleh Badan Pusat Statistik (BPS), tercatat sebesar 0,54 persen (mtm). Tingkat inflasi tahun kalender (Januari-Juni) 2015 sebesar 0,96 persen dan tingkat inflasi tahun ke tahun (Juni 2015 terhadap Juni 2014) sebesar 7,26 persen.Menurut Agus, memang terdapat indikasi penurunan daya beli masyarakat selama enam bulan pertama tahun ini."Pengaruh daya beli agak lemah mungkin ada. Kita lihat, betul-betul ekonomi 2015 ini dijaga di semester kedua," kata Agus.Sementara itu, terkait inflasi Juni, Deputi Gubernur BI Perry